In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import sys

In [2]:
train = pd.read_csv(r'E:\data\kaggle\spookyauthor\train.csv',index_col=False)
test = pd.read_csv(r'E:\data\kaggle\spookyauthor\test.csv',index_col=False)

In [3]:
train.text += '\\'
#import unidecode
#decoder = unidecode.unidecode
train.text = train.text.str.lower() #apply(decoder)
t0 = train[train.author==train.author.unique()[0]].text
t0.reset_index(drop=True,inplace=True)

In [4]:
text = train.text.sum()
text += test.text.str.lower().sum()
chars = sorted(list(set(text)))
print(chars,len(chars))

[' ', '"', "'", ',', '.', ':', ';', '?', '\\', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'à', 'â', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'î', 'ï', 'ñ', 'ô', 'õ', 'ö', 'û', 'ü', 'α', 'δ', 'ν', 'ο', 'π', 'ς', 'υ', 'ἶ'] 61


In [193]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

input_size = len(chars)
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 20
step = 1
sentences = []
next_chars = []
def linetotensor(line):
    l = len(line)
    line += '\\'
    sentence = line[: l-1]
    next_char = line[1: l]
    x = np.zeros((l, 1, input_size))
    y = np.zeros((l,1))
    for t,letter in enumerate(sentence):
        x[t, 0, char_indices[letter]] = 1
    for t,letter in enumerate(next_char):
        y[t,0 ] = char_indices[letter]
    return x,y

def strtotensor(string):
    string = string.lower()
    l = len(string)
    x = np.zeros((l, 1, input_size))
    
    for t,letter in enumerate(string):
        x[t, 0, char_indices[letter]] = 1
    return x

def paddedtensor(part):
    num_lines = len(part)
    maxseq = part.str.len().max()
    X = np.zeros((maxseq, num_lines, 61))
    Y = np.zeros((maxseq,num_lines))
    L = np.zeros(num_lines)
    W = np.zeros_like(Y)
    for index in range(num_lines):
        line = part.iloc[index]
        l = len(line)
        line += '\\'
        sentence = line[: l-1]
        next_char = line[1: l]       
        for t,letter in enumerate(sentence):
            X[t, index, char_indices[letter]] = 1
        for t,letter in enumerate(next_char):
            Y[t,index ] = char_indices[letter]
        L[index]  = l
        W[:l,index] = 1
    return X,Y,L,W

In [258]:
p_1 = t0.str.len().quantile(0.25)
p_2 = t0.str.len().quantile(0.5)
p_3 = t0.str.len().quantile(0.75)
p_4 = t0.str.len().quantile(1)
p1 = t0[t0.str.len()<p_1]
x,y,l,w = paddedtensor(p1)

In [295]:
tf.reset_default_graph()

epoch_num = 50
sample_size = len(p1)
batch_size = 1000
rnn_size = 256



rnn = tf.contrib.rnn.LSTMCell(rnn_size,activation=tf.sigmoid,
                              use_peepholes=True)
#rnn2 = tf.contrib.rnn.LSTMCell(rnn_size,activation=tf.sigmoid,
                              #use_peepholes=True)
                             
#rnn = tf.contrib.rnn.MultiRNNCell([rnn1,rnn2])

#input and label

X = tf.placeholder(tf.float32,shape=[None,None,input_size])
Y = tf.placeholder(tf.int32,shape=[None,None])
L = tf.placeholder(tf.int32,shape=[None,])

W = tf.Variable(tf.random_normal([rnn_size,input_size]))+1
b = tf.Variable(tf.random_normal([input_size])) +1

weight = tf.placeholder(tf.float32,shape=[None,None])
#initial_state = rnn.zero_state(None, dtype=tf.float32)
output,state = tf.nn.dynamic_rnn(rnn,X,
                                 dtype = tf.float32,
                                 sequence_length = L,
                                time_major=True)

logits = tf.tensordot(output,W,[[2],[0]])+b
prob = tf.nn.softmax(logits)
loss = tf.losses.sparse_softmax_cross_entropy(labels = Y,
                                              logits = logits,
                                              reduction='none')
loss *= weight
LOSS = tf.reduce_sum(loss)
LOSS_ = tf.reduce_mean(loss)
      

session =tf.Session()
session.run(tf.global_variables_initializer())
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(LOSS_)
for epoch in range(epoch_num):
    loss_epoch=0
    iter_ = 0
    permutation = np.random.permutation(sample_size)
    x = x[:,permutation,:]
    y = y[:,permutation]
    l = l[permutation]
    w = w[:,permutation]
    for i in range(sample_size//batch_size):
        s = i * batch_size
        e = s + batch_size
        x_ = x[:,s:e,:]
        y_ = y[:,s:e]
        l_ = l[s:e]
        
        w_ = w[:,s:e]
        
        out_loss,p,total_loss = session.run((loss,prob,LOSS),
                          feed_dict={X:x_,Y:y_,L:l_,weight:w_})
        a = session.run(train_step,feed_dict={X:x_,Y:y_,L:l_,weight:w_})
        
        
        loss_epoch += total_loss
    print(epoch,':',(loss_epoch/92036))


0 : 7.27845761441
1 : 6.38023029032
2 : 5.96327048655
3 : 5.64576972598
4 : 5.2953803403
5 : 5.08410805283
6 : 4.92845530553
7 : 4.71558717513
8 : 4.49286794026
9 : 4.2640318734
10 : 4.14777912991
11 : 3.97313517265
12 : 3.76888425453
13 : 3.66408457017
14 : 3.50895573471
15 : 3.41366353927
16 : 3.28732846386
17 : 3.22163134534
18 : 3.15770459385
19 : 3.08005664903
20 : 3.05781433352
21 : 2.96579327654
22 : 2.90845905407
23 : 2.86824890532
24 : 2.82741552898
25 : 2.75337656596
26 : 2.69679171058
27 : 2.66506312747
28 : 2.60672624028
29 : 2.57320182456
30 : 2.54730570782
31 : 2.49991087047
32 : 2.46813237483
33 : 2.4191834581
34 : 2.38747236136
35 : 2.37895497414
36 : 2.30814097201
37 : 2.3243494733
38 : 2.25122676181
39 : 2.25293703008
40 : 2.2399855763
41 : 2.22465292114
42 : 2.19677269628
43 : 2.18751731659
44 : 2.14591810542
45 : 2.16535360484
46 : 2.13377696635
47 : 2.12062654152
48 : 2.10655375342
49 : 2.08491378374


In [267]:
out_loss

array([[    0.        ,  1185.96972656,  1385.63696289, ...,
            0.        ,   829.644104  ,  1185.96972656],
       [ 1453.96838379,   256.22143555,    25.80932617, ...,
         1659.28637695,  1663.75805664,   256.22143555],
       [ 1479.2791748 ,    15.0725708 ,  1724.37866211, ...,
         1479.2791748 ,    15.07250977,    15.0725708 ],
       ..., 
       [    0.        ,     0.        ,     0.        , ...,
            0.        ,     0.        ,     0.        ],
       [    0.        ,     0.        ,     0.        , ...,
            0.        ,     0.        ,     0.        ],
       [    0.        ,     0.        ,     0.        , ...,
            0.        ,     0.        ,     0.        ]], dtype=float32)

In [ ]:

seed = 'nono '
sys.stdout.write(seed)
for i in range(555):
    x_ = strtotensor(seed)
    l_ = [len(seed)]
    result = session.run(prob,
                          feed_dict={X:x_,L:l_})
    letter_ind = np.argmax(result[-1,0,:])
    letter = indices_char[letter_ind]
    seed +=letter
    sys.stdout.write(letter)
    #print(result.shape)

nono n                                                                                                                                                                                                         

# KERAS USELESS

In [45]:
from keras.models import Sequential
from keras.layers import Dense, Activation,TimeDistributed
from keras.layers import LSTM,LSTMCell
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file

import random
import sys

In [ ]:
text = t0.sum()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 20
step = 1
sentences = []
next_chars = []
for i in range(5):
    line = t0[i]
    l = len(line)
    sentence = line[: l-1]
    next_char = line[1: l]
    x = np.zeros((l, 1, len(chars)), dtype=np.bool)
    y = np.zeros((l,1, len(chars)), dtype=np.bool)
    for t,letter in enumerate(sentence):
        x[t, 0, char_indices[letter]] = 1
    for t,letter in enumerate(next_char):
        y[t, 0, char_indices[letter]] = 1
    sentences.append(x)
    next_chars.append(y)
print('nb sequences:', len(sentences))

print('Vectorization...')
'''
for i  in range(len(sentences)):
    line = sentences[i]
    l = len(line)
    
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
'''

# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(None, len(chars))))
model.add((Dense(len(chars))))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(len(sentences)):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    x = sentences[iteration]
    y = next_chars[iteration]
    model.fit(x, y,
              batch_size=None,
              epochs=1,verbose=0)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()